# Financial Data and API Calls

 - Darlene Adams

## Imports and Logins

In [1]:
# Import packages
import os, time, json
import tmdbsimple as tmdb 
import pandas as pd
from tqdm.notebook import tqdm_notebook
# Create the folder for saving files (if it doesn't exist)
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'create_movies_database.sql',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'movies_erd.mwb',
 'movies_erd.mwb.bak',
 'Movies_ERD.png',
 'Part_One.csv',
 'Part_One_Final.csv',
 'ratings_filtered.csv',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'titles_filtered.csv',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json',
 'wrong.ipynb']

In [2]:
# Load API Credentials
with open('/Users/dmusl/.secret/yelp_api.json') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# setting the api key
tmdb.API_KEY =  login['api-key']

# Custom Functions

In [4]:
def get_movie_with_rating(movie_id): 
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)



# Create the Outer Loop

In [8]:
# Define list of years to get from API for outer loop
YEARS_TO_GET = [2001,2002]

In [9]:
basics = pd.read_csv('Data/titles_filtered.csv')
basics = basics.drop(columns=['Unnamed: 0'])
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
86974,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
86975,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
86976,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
86977,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [ ]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    # Create an empty list for saving errors
    errors = []
    
    # Define the JSON file to store results for the year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # Check if the JSON file exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it does not exist: create it
    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for year={YEAR}.")
    
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    # If it exists, print a message
    else:
        print(f'The file {JSON_FILE} already exists.')

    # Filtering for movies from selected startYear
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst']
    movie_ids.head()

    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    previous_df

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    # Loop through movie_ids_to_get with a tqdm progress bar
    for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):

        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #pre-made function
            # Append/extend results to existing file
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        # If it fails, make a dict with just the id and None for certification.
        except Exception as e:
            errors.append([movie_id, e])

    print(f"- Total errors: {len(errors)}")

    # Save the final results to a csv.gz file
    final_year_df = pd.read_json(JSON_FILE)

    csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
    final_year_df.to_csv(csv_fname, compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

The file Data/tmdb_api_results_2001.json already exists.


Movies from 2001:   0%|          | 0/1576 [00:00<?, ?it/s]

- Total errors: 1576
The file Data/tmdb_api_results_2002.json already exists.


Movies from 2002:   0%|          | 0/1572 [00:00<?, ?it/s]

In [ ]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    # Create an empty list for saving errors
    errors = []

    # Define the JSON file to store results for the year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # Check if the JSON file exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it does not exist: create it
    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for year={YEAR}.")
    
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    # If it exists, print a message
    else:
        print(f'The file {JSON_FILE} already exists.')


In [ ]:
# Filtering for movies from selected startYear
df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
movie_ids = df['tconst']
movie_ids.head()

In [ ]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)
previous_df

In [ ]:
 # filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

# Create the Inner Loop

In [70]:
# Loop through movie_ids_to_get with a tqdm progress bar
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):
    # Attempt to retrieve then data for the movie id
    try:
        temp = get_movie_with_rating(movie_id)  
        # Append/extend results to existing file
        write_json(temp,JSON_FILE)
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
        
    except Exception as e:
        errors.append([movie_id, e])


Movies from 2002:   0%|          | 0/1572 [00:00<?, ?it/s]

In [66]:
 print(f"- Total errors: {len(errors)}")

- Total errors: 1572


In [67]:
# Save the final results to a csv.gz file
final_year_df = pd.read_json(JSON_FILE)

csv_years = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
final_year_df.to_csv(csv_years, compression="gzip", index=False)